In [1]:
import cv2
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.layers as layers
from data_generator import DataGenerator
import pandas as pd
from __future__ import absolute_import, division, print_function
import os
import sys
print(os.getcwd())
os.environ["CUDA_VISIBLE_DEVICES"]= sys.argv[1]
import tensorflow as tf
#tf.compat.v1.enable_eager_execution()
import numpy as np
#from keras.datasets import cifar10
import keras.callbacks as callbacks
#import keras.utils.np_utils as kutils
from tensorflow.keras import utils
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import plot_model
#from wide_resnet import WRNModel
from keras import backend as K
from keras.callbacks import ModelCheckpoint, CSVLogger
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import h5py
#from keras.utils import plot_model
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
tf.compat.v1.disable_eager_execution()
from keras.utils import to_categorical
import keras
# parts of the code were adapted from https://github.com/jemiar/surgery-gesture-recog
# parts of the code were adapted from https://github.com/yashkant/padam-tensorflow

/Users/boabangfrancis
Instructions for updating:
non-resource variables are not supported in the long term


In [8]:
pwd

'C:\\Users\\F_BOABA'

In [2]:
sys.path.append(os.path.dirname(os.getcwd()))
sys.path.append(os.getcwd())

In [3]:
base_directory = '/Users/boabangfrancis/'

In [ ]:
import zipfile
with zipfile.ZipFile("/Users/boabangfrancis/Suturing.zip","r") as zip_ref:
    zip_ref.extractall("/Users/boabangfrancis/Suturing")

In [4]:
import os
#h=[]
#for i in os.listdir('/Users/boabangfrancis/transcription/transcription/'):
#    if i.startswith('S'):
 #       h.append(i)
#transcriptions=sorted(h[16:17])
#transcriptions=['Suturing_B001.csv', 'Suturing_C001.csv','Suturing_D001.csv','Suturing_E001.csv', 'Suturing_F001.csv','Suturing_G001.csv','Suturing_H001.csv','Suturing_I001.csv']
transcriptions=['Suturing_D001.csv', 'Suturing_D002.csv','Suturing_D003.csv','Suturing_D004.csv', 'Suturing_D005.csv','Suturing_E001.csv','Suturing_E002.csv','Suturing_E003.csv','Suturing_E004.csv','Suturing_E005.csv']
#transcriptions=['Suturing_D001.csv']

In [5]:
def save_gesture(fromarray, height=240, width=320, folder='data_action\\', idnumber=1, base_directory=''):
    data = []
    labels = {}
    video_directory = os.path.join(base_directory, 'Suturing', 'Suturing', 'video')

    for arr in fromarray:
        video_file = os.path.join(video_directory, arr[:-4] + '_capture1.avi')
        cap = cv2.VideoCapture(video_file)
        i = 1
        red_frames = np.empty((0, height, width))
        green_frames = np.empty((0, height, width))
        blue_frames = np.empty((0, height, width))

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            if i % 3 == 1:
                frame = cv2.resize(frame, (width, height), interpolation=cv2.INTER_AREA)
                f = np.asarray(frame)
                f = cv2.cvtColor(f, cv2.COLOR_BGR2RGB)
                red_frames = np.append(red_frames, np.expand_dims(f[:,:,0], axis=0), axis=0)
                green_frames = np.append(green_frames, np.expand_dims(f[:,:,1], axis=0), axis=0)
                blue_frames = np.append(blue_frames, np.expand_dims(f[:,:,2], axis=0), axis=0)
            i += 1
        cap.release()

        red_frames = red_frames / 255.0
        green_frames = green_frames / 255.0
        blue_frames = blue_frames / 255.0

        df = pd.read_csv(os.path.join(base_directory + 'transcription' + arr[:-4] + '.csv')) 
        arr_gesture = np.array(df)

        for t in arr_gesture[10:-4]:
            left = (t[0] + 1) // 3
            right = (t[1] - 1) // 3 + 1
            w = t[2]
            c = w[-1:]
            num_blocks = (right - left + 1) // 10
            for index in range(num_blocks):
                block = np.concatenate([
                    red_frames[left+index*10:left+(index+1)*10,:,:],
                    green_frames[left+index*10:left+(index+1)*10,:,:],
                    blue_frames[left+index*10:left+(index+1)*10,:,:]], axis=2)
                npy_name = 'id_' + str(idnumber)
                temp_obj = {'id': npy_name, 'file': arr, 'gesture': c}
                data.append(npy_name)
                labels[npy_name] = c
                np.save(os.path.join(base_directory, folder, npy_name + '.npy'), block)
                idnumber += 1

    return data, labels

In [6]:
def create_model(height=240, width=320):
    # shape of input: variable number frames x height x width x 3 channels (RGB)
    input = tf.keras.Input((10, height, width, 3))

    # 1st ConvLSTM block includes Conv2D with 8 filters, MaxPool2D and BatchNormalization
    # Using keras TimeDistributed layer to apply Conv2D, MaxPool2D and BatchNormalization to each frame
    conv2D_1 = layers.Conv2D(filters=8, kernel_size=(3,3), activation='relu')
    x = layers.TimeDistributed(conv2D_1)(input)
    maxpool_1 = layers.MaxPool2D(pool_size=(2,2))
    x = layers.TimeDistributed(maxpool_1)(x)
    batchnorm_1 = layers.BatchNormalization()
    x = layers.TimeDistributed(batchnorm_1)(x)

    # 2nd ConvLSTM block includes Conv2D with 16 filters, MaxPool2D and BatchNormalization
    # Using keras TimeDistributed layer to apply Conv2D, MaxPool2D and BatchNormalization to each frame
    conv2D_2 = layers.Conv2D(filters=16, kernel_size=(3,3), activation='relu')
    x = layers.TimeDistributed(conv2D_2)(x)
    maxpool_2 = layers.MaxPool2D(pool_size=(2,2))
    x = layers.TimeDistributed(maxpool_2)(x)
    batchnorm_2 = layers.BatchNormalization()
    x = layers.TimeDistributed(batchnorm_2)(x)
    
    
    
    # 3nd ConvLSTM block includes Conv2D with 32 filters, MaxPool2D and BatchNormalization
    # Using keras TimeDistributed layer to apply Conv2D, MaxPool2D and BatchNormalization to each frame
    conv2D_3 = layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu')
    x = layers.TimeDistributed(conv2D_3)(x)
    maxpool_3 = layers.MaxPool2D(pool_size=(2,2))
    x = layers.TimeDistributed(maxpool_3)(x)
    batchnorm_3 = layers.BatchNormalization()
    x = layers.TimeDistributed(batchnorm_3)(x)
    
    
        
    # 4nd ConvLSTM block includes Conv2D with 16 filters, MaxPool2D and BatchNormalization
    # Using keras TimeDistributed layer to apply Conv2D, MaxPool2D and BatchNormalization to each frame
    conv2D_4 = layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu')
    x = layers.TimeDistributed(conv2D_4)(x)
    maxpool_4 = layers.MaxPool2D(pool_size=(2,2))
    x = layers.TimeDistributed(maxpool_4)(x)
    batchnorm_4 = layers.BatchNormalization()
    x = layers.TimeDistributed(batchnorm_4)(x)
    
    
    # 5nd ConvLSTM block includes Conv2D with 128 filters, MaxPool2D and BatchNormalization
    # Using keras TimeDistributed layer to apply Conv2D, MaxPool2D and BatchNormalization to each frame
    conv2D_5 = layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu')
    x = layers.TimeDistributed(conv2D_5)(x)
    maxpool_5 = layers.MaxPool2D(pool_size=(2,2))
    x = layers.TimeDistributed(maxpool_5)(x)
    batchnorm_5 = layers.BatchNormalization()
    x = layers.TimeDistributed(batchnorm_5)(x)
    
    

    # Flatten x to supply it to LSTM layer
    flatten = layers.Flatten()
    x = layers.TimeDistributed(flatten)(x)

    # LSTM layer with 8 output units at the last LSTM block
    x = layers.LSTM(8, return_sequences=False)(x)
    # Fully-connected layer with 32 units and L2 regularization
    x = layers.Dense(units=4, activation='relu', kernel_regularizer=keras.regularizers.L2(0.00001))(x)

    # output shape (10,)
    output = layers.Dense(units=10, activation='softmax')(x)

    model = tf.keras.Model(input, output, name='ConvLSTM')
    return model

Create model

In [7]:
# Create data generator for training and validation

data, labels =save_gesture(fromarray=transcriptions, height=240, width=320, folder='data_action/', idnumber=1)

In [41]:
optim_params = {
    
    
    'proposed_ASGD': {
        'weight_decay': 0.0001,
        'lr_min_d': 0.00001,
        'lr_max_d':0.0001,
        #'lr': 0, # dummy value
        'b1': 0.9,
        'b2': 0.999,
        'color': 'red',
        'linestyle':'-'
    },
    
    
    'proposed_ASGD_adam_version': {
        'weight_decay': 0.00001,
        'lr_min_d': 0.00001,
        'lr_max_d':0.0001,
        #'lr': 0, # dummy value
        'b1': 0.9,
        'b2': 0.999,
        'color': 'red',
        'linestyle':'-'
    },
    
    'padam': {
        'weight_decay': 0.0001,
        'lr_max': 0.01,
        'p': 0.125,
        'b1': 0.9,
        'b2': 0.999,
        'color': 'darkred',
        'linestyle':'-'
    },
    'adam': {
        'weight_decay': 0.0001,
        'lr_min': 0.00001,
        'b1': 0.9,
        'b2': 0.99,
        'color': 'orange',
        'linestyle':'--'
    },
    'adamw': {
        'weight_decay': 0.025,
        'lr_min': 0.00001,
        'b1': 0.9,
        'b2': 0.99,
        'color': 'magenta',
        'linestyle':'--'
    },
    'amsgrad': {
        'weight_decay': 0.0001,
        'lr_min': 0.00001,
        'b1': 0.9,
        'b2': 0.99,
        'color' : 'darkgreen',
        'linestyle':'-.'
    },
    'sgd': {
        'weight_decay': 0.0005,
        'lr_max': 0.001,
        'm': 0.9,
        'color': 'blue',
        'linestyle':'-'
    }
}

In [20]:
from sklearn.utils import resample
#ham_messages=[{'id_1','id_1','id_1','id_1','id_3','id_1','id_3','id_2','id_1','id_1','id_4','id_1','id_1'}]
ham_downsample = resample([labels],
             replace=True,
             n_samples=len([labels]),
             random_state=42)

In [25]:
ham_downsample

In [ ]:
def save_model(filepath, model):
    file = h5py.File(filepath,'w')
    weight = model.get_weights()
    for i in range(len(weight)):
        file.create_dataset('weight'+str(i),data=weight[i])
    file.close()

In [ ]:
def load_model(filepath, model):
    file=h5py.File(filepath,'r')
    weight = []
    for i in range(len(file.keys())):
        weight.append(file['weight'+str(i)][:])
    model.set_weights(weight)
    return model 

In [38]:
#dimension=np.load('/Users/boabangfrancis/data_action/' + train)
#dim_shape=(dimension.shape[0],dimension.shape[1], dimension.shape[2])
#labelssorted=dict(sorted(labels.items()))
params = {'dim': (10, 240, 320),
          'batch_size': 1,
          'n_classes': 10,
          'n_channels': 3,
          'folder': 'data_action/',
          'shuffle': True}
  
#y = tf.keras.layers.UpSampling3D(size=10)(normals)   
    
train_generator = DataGenerator(data[21:], ham_downsample, **params)
val_generator = DataGenerator(data[1:20], labels, **params)

In [181]:
from sklearn.utils import resample
#ham_messages=[{'id_1','id_1','id_1','id_1','id_3','id_1','id_3','id_2','id_1','id_1','id_4','id_1','id_1'}]
ham_downsample_u= resample([ham_downsample],
             replace=True,
             n_samples=len([ham_downsample]),
             random_state=100)

In [42]:
from padam import Padam
from amsgrad import AMSGrad
from proposed_ASGD import Proposed_ASGD
from proposed_ASGD_adam_version import  Proposed_ASGD_adam_version
#learning_rate=0.00001
#learning_rate_min=0.00001
#learning_rate_max=0.001
#optim_array=[ 'padam', 'proposed_ASGD_adam_version', 'proposed_ASGD',  'adam', 'amsgrad']
#optim_array=[]
optim_array=['padam']
#optim_array=['padam']
history={}
for optimizer in optim_array:
    op=optim_params[optimizer]
    
    logfile= 'log_' + optimizer +'_ExpectGoodResult' + '.csv'
    #op['lr']=op['lr']
   # learning_rate_min=op['lr_min']
    
    if optimizer == 'proposed_ASGD':
        learning_rate_max=op['lr_max_d']/1
        learning_rate_min=op['lr_min_d']/1
        optim = Proposed_ASGD(learning_rate_min=learning_rate_min,learning_rate_max=learning_rate_max, beta1=op['b1'], beta2=op['b2'])
    elif optimizer == 'proposed_ASGD_adam_version':
        learning_rate_max=op['lr_max_d']/1
        learning_rate_min=op['lr_min_d']/1
        optim = Proposed_ASGD_adam_version(learning_rate_min=learning_rate_min,learning_rate_max=learning_rate_max, beta1=op['b1'], beta2=op['b2'])
            
    elif optimizer == 'padam':
        learning_rate_max=op['lr_max']/1
        optim = Padam(learning_rate=learning_rate_max, p=op['p'], beta1=op['b1'], beta2=op['b2'])
    elif optimizer == 'adam':
        learning_rate_min=op['lr_min']/1
        optim = tf.train.AdamOptimizer(learning_rate=learning_rate_min, beta1=op['b1'], beta2=op['b2'])
    elif optimizer == 'adamw':
        learning_rate_min=op['lr_min']/1
            # adamw = tf.contrib.opt.extend_with_decoupled_weight_decay(tf.train.AdamOptimizer)
        optim = tf.contrib.opt.AdamWOptimizer(weight_decay=op['weight_decay'], learning_rate=learning_rate_min,  beta1=op['b1'], beta2=op['b2'])
    elif optimizer == 'amsgrad':
        learning_rate_min=op['lr_min']/1
        optim = AMSGrad(learning_rate=learning_rate_min, beta1=op['b1'], beta2=op['b2'])
    elif optimizer == 'sgd':
        learning_rate_max=op['lr_max']/1
        optim = tf.train.MomentumOptimizer(learning_rate=learning_rate_max, momentum=op['m'])
    model = create_model(240, 320)  
    model.compile(optimizer=optim, loss='categorical_crossentropy', metrics=['accuracy', 'top_k_categorical_accuracy'])
    csv_logger = CSVLogger(logfile, append=True, separator=';')
        #model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(), optimizer=keras.optimizers.SGD(learning_rate=learning_rate), metrics=metrics)
    model.fit_generator(generator=train_generator, validation_data=val_generator, epochs=200, shuffle=True, verbose=1, callbacks = [csv_logger])
    scores = model.evaluate_generator(val_generator, verbose=1)
    print("Final test loss and accuracy:", scores)
    #save_model(save_model_filepath, model)

Epoch 1/200


/var/folders/ql/5nmrkn3145n9_pzr7h7w7_gh0000gn/T/ipykernel_1096/3508825634.py:49: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator=train_generator, validation_data=val_generator, epochs=200, shuffle=True, verbose=1, callbacks = [csv_logger])


175/175 [==============================] - 13s 69ms/step - loss: 2.0700 - accuracy: 0.2686 - top_k_categorical_accuracy: 0.8114 - val_loss: 2.3666 - val_accuracy: 0.0526 - val_top_k_categorical_accuracy: 0.5263
Epoch 2/200
175/175 [==============================] - 12s 67ms/step - loss: 1.6036 - accuracy: 0.4800 - top_k_categorical_accuracy: 0.9200 - val_loss: 1.9697 - val_accuracy: 0.2632 - val_top_k_categorical_accuracy: 0.7895
Epoch 3/200
175/175 [==============================] - 12s 67ms/step - loss: 1.3794 - accuracy: 0.4629 - top_k_categorical_accuracy: 0.9829 - val_loss: 1.7334 - val_accuracy: 0.3158 - val_top_k_categorical_accuracy: 1.0000
Epoch 4/200
175/175 [==============================] - 12s 68ms/step - loss: 1.2439 - accuracy: 0.5200 - top_k_categorical_accuracy: 1.0000 - val_loss: 1.6124 - val_accuracy: 0.4211 - val_top_k_categorical_accuracy: 0.9474
Epoch 5/200
175/175 [==============================] - 12s 69ms/step - loss: 1.1122 - accuracy: 0.5714 - top_k_categoric

Epoch 38/200
175/175 [==============================] - 12s 68ms/step - loss: 0.6196 - accuracy: 0.7829 - top_k_categorical_accuracy: 1.0000 - val_loss: 3.6790 - val_accuracy: 0.3158 - val_top_k_categorical_accuracy: 0.8421
Epoch 39/200
175/175 [==============================] - 12s 68ms/step - loss: 0.6398 - accuracy: 0.7771 - top_k_categorical_accuracy: 1.0000 - val_loss: 3.8058 - val_accuracy: 0.3158 - val_top_k_categorical_accuracy: 0.9474
Epoch 40/200
175/175 [==============================] - 12s 68ms/step - loss: 0.4447 - accuracy: 0.8457 - top_k_categorical_accuracy: 1.0000 - val_loss: 4.7025 - val_accuracy: 0.3684 - val_top_k_categorical_accuracy: 0.7895
Epoch 41/200
175/175 [==============================] - 12s 68ms/step - loss: 0.6001 - accuracy: 0.7886 - top_k_categorical_accuracy: 0.9943 - val_loss: 4.3152 - val_accuracy: 0.2632 - val_top_k_categorical_accuracy: 0.8947
Epoch 42/200
175/175 [==============================] - 12s 68ms/step - loss: 0.5588 - accuracy: 0.8229 

175/175 [==============================] - 12s 67ms/step - loss: 0.7720 - accuracy: 0.6914 - top_k_categorical_accuracy: 1.0000 - val_loss: 6.6068 - val_accuracy: 0.3158 - val_top_k_categorical_accuracy: 0.7895
Epoch 111/200
175/175 [==============================] - 12s 66ms/step - loss: 0.6868 - accuracy: 0.7543 - top_k_categorical_accuracy: 1.0000 - val_loss: 5.3284 - val_accuracy: 0.2632 - val_top_k_categorical_accuracy: 0.8947
Epoch 112/200
175/175 [==============================] - 12s 67ms/step - loss: 0.6036 - accuracy: 0.7829 - top_k_categorical_accuracy: 1.0000 - val_loss: 8.2775 - val_accuracy: 0.2105 - val_top_k_categorical_accuracy: 0.7895
Epoch 113/200
175/175 [==============================] - 12s 67ms/step - loss: 0.6941 - accuracy: 0.7543 - top_k_categorical_accuracy: 1.0000 - val_loss: 5.1844 - val_accuracy: 0.1579 - val_top_k_categorical_accuracy: 0.9474
Epoch 114/200
175/175 [==============================] - 12s 66ms/step - loss: 0.7029 - accuracy: 0.7371 - top_k_c

175/175 [==============================] - 12s 66ms/step - loss: 0.6244 - accuracy: 0.7943 - top_k_categorical_accuracy: 1.0000 - val_loss: 6.2153 - val_accuracy: 0.2105 - val_top_k_categorical_accuracy: 0.8947
Epoch 183/200
175/175 [==============================] - 12s 66ms/step - loss: 0.4882 - accuracy: 0.8171 - top_k_categorical_accuracy: 1.0000 - val_loss: 5.0709 - val_accuracy: 0.3158 - val_top_k_categorical_accuracy: 1.0000
Epoch 184/200
175/175 [==============================] - 12s 66ms/step - loss: 0.5220 - accuracy: 0.8286 - top_k_categorical_accuracy: 1.0000 - val_loss: 5.0570 - val_accuracy: 0.3158 - val_top_k_categorical_accuracy: 1.0000
Epoch 185/200
175/175 [==============================] - 11s 65ms/step - loss: 0.4567 - accuracy: 0.8514 - top_k_categorical_accuracy: 1.0000 - val_loss: 3.5881 - val_accuracy: 0.4211 - val_top_k_categorical_accuracy: 0.8947
Epoch 186/200
175/175 [==============================] - 12s 67ms/step - loss: 0.3877 - accuracy: 0.8914 - top_k_c

/var/folders/ql/5nmrkn3145n9_pzr7h7w7_gh0000gn/T/ipykernel_1096/3508825634.py:50: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  scores = model.evaluate_generator(val_generator, verbose=1)


19/19 [==============================] - 0s 16ms/step - loss: 5.0054 - accuracy: 0.2632 - top_k_categorical_accuracy: 1.0000
Final test loss and accuracy: [5.0054030418396, 0.2631579041481018, 1.0]
